In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
user_artist_data = pd.read_csv('/content/drive/MyDrive/SML/song_data/user_artists.dat', sep='\t')

# Load the artist data
artist_data = pd.read_csv('/content/drive/MyDrive/SML/song_data/artists.dat', sep='\t', usecols=['id', 'name'])

In [ ]:
user_artist_data.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
artist_data.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


In [ ]:
data = pd.merge(user_artist_data, artist_data, left_on='artistID', right_on='id')

# Keep only relevant columns
data = data[['userID', 'name', 'weight']]

reader = Reader(rating_scale=(1, 100))

# Load the data into Surprise Dataset format
data = Dataset.load_from_df(user_artist_data[['userID', 'artistID', 'weight']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Define the SVD model
model = SVD(n_factors=50, reg_all=0.1, verbose=True)

# Train the model on the training set
model.fit(trainset)

# Make predictions on the testing set
predictions = model.test(testset)


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [ ]:
testset[6]

(2076, 207, 88.0)

In [ ]:
predictions = model.test(testset)

In [ ]:
print(predictions)

[Prediction(uid=604, iid=1729, r_ui=962.0, est=100, details={'was_impossible': False}), Prediction(uid=132, iid=1009, r_ui=99.0, est=100, details={'was_impossible': False}), Prediction(uid=1155, iid=618, r_ui=298.0, est=100, details={'was_impossible': False}), Prediction(uid=127, iid=920, r_ui=560.0, est=100, details={'was_impossible': False}), Prediction(uid=1310, iid=10184, r_ui=4.0, est=100, details={'was_impossible': False}), Prediction(uid=52, iid=1565, r_ui=93.0, est=100, details={'was_impossible': False}), Prediction(uid=2076, iid=207, r_ui=88.0, est=100, details={'was_impossible': False}), Prediction(uid=520, iid=1406, r_ui=209.0, est=100, details={'was_impossible': False}), Prediction(uid=1075, iid=5149, r_ui=1152.0, est=100, details={'was_impossible': False}), Prediction(uid=1628, iid=5149, r_ui=254.0, est=100, details={'was_impossible': False}), Prediction(uid=1690, iid=4704, r_ui=835.0, est=100, details={'was_impossible': False}), Prediction(uid=468, iid=5425, r_ui=292.0, e

In [ ]:
accuracy = sum([int(pred.est >= 50) == int(pred.r_ui >= 50) for pred in predictions]) / len(predictions)

print('Accuracy:', accuracy)

Accuracy: 0.8700920988851187


In [ ]:
user_id = 2076
user_items = user_artist_data[user_artist_data['userID'] == user_id]
user_items = user_items.merge(artist_data, left_on='artistID', right_on='id')
user_items = user_items.drop(['id'], axis=1)
user_items = user_items.rename(columns={'name': 'artist'})
user_items = user_items[['userID', 'artist', 'weight']]
user_items['Estimate_Score'] = user_items['artist'].apply(lambda x: model.predict(user_id, artist_data[artist_data['name'] == x]['id'].item()).est)
user_items = user_items.sort_values('Estimate_Score', ascending=False)
print(user_items.tail())

    userID           artist  weight  Estimate_Score
19    2076  Capital Inicial     113             100
20    2076        Nightwish     152             100
21    2076            Epica     102             100
22    2076         t.A.T.u.     164             100
49    2076    Madame Saatan     118             100


In [ ]:
user_items['artist'].count()

50

In [ ]:
data = pd.merge(user_artist_data, artist_data, left_on='artistID', right_on='id')

In [ ]:
common_elements = user_items['artist'].isin(data[data.userID==604]["name"])

# Count the number of common elements
num_common_elements = common_elements.sum()

print(f"Number of common elements: {num_common_elements}")

Number of common elements: 50


In [ ]:
data[data.userID==604].count()

userID      50
artistID    50
weight      50
id          50
name        50
dtype: int64